In [1]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 50)  # Показывать до 50 колонок
pd.set_option('display.max_colwidth', 100)  # Показывать до 50 колонок

In [17]:
data = pd.read_excel("../cuprum_3.xlsx", sheet_name="Лист4") #cuprum/Лист4/Лист2/Лист1
data.rename(columns={"пол":"gender", "возраст":"age"}, inplace=True)
data = data[data['action_type'] == 'CLICKED']
data.drop(columns=["esb_ehr_id","patientnet_ehr_id", "medialog_ehr_id","action_type"], inplace=True)
data

,id,article_id,ehr_id,created_at,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
0,3744933,61c03691a7568f6cda4a0c95,961420,2023-07-01,2,35,1990-05-29,Чем опасна трипофобия,https://cuprum.media/science-answers/trypophobia?from=avaapp,795.0,2021-12-20,longread,0,Ответили по науке
1,3745298,5f22ce85e24e57105a57d2a5,100445,2023-07-01,1,37,1987-09-26,Мирамистин и хлоргексидин можно использовать вместо санитайзера?,https://cuprum.media/proverka-sluha/miramistin-i-hlorgeksidin-mozhno-ispolzovat-vmesto-sanitajze...,4005.0,2020-09-05,longread,"Пандемия,Профилактика,Патогены,Вирусы",Проверка слуха
2,3745310,5fdba88d08d6334b9536bd71,491804,2023-07-01,1,47,1978-01-07,Открываем окна: кому и как нужно проветривать квартиру,https://cuprum.media/science-answers/how-to-ventilate-the-apartment?from=avaapp,554.0,2020-12-17,longread,0,Ответили по науке
3,3745453,63ea42aaa6594aa87e07c4f2,65721,2023-07-01,1,53,1972-02-03,"Правда ли, что лампы для сушки ногтей вызывают рак",https://cuprum.media/science-answers/nail-lamp-danger?from=avaapp,8032.0,2023-02-13,longread,"Вредное,Косметика,Булшит",Ответили по науке
4,3745512,608a88b3b587d22877730773,1281788,2023-07-01,2,74,1951-04-21,Можно ли забеременеть от двух мужчин одновременно,https://cuprum.media/stesnyayus-sprosit/superfecundation?from=avaapp,16787.0,2021-04-29,longread,"Репродукция,Булшит,Беременность",Стесняюсь спросить
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30835,9608721,63a9765bff95853c500cce62,97731,2024-07-30,1,49,1976-05-09,Почему возникают ошибки в результатах анализов,https://cuprum.media/science-answers/lab-test-mistakes?from=avaapp,5907.0,2023-01-04,longread,"Анализы,Диагностика",Ответили по науке
30836,9608800,606dd15f70bc473f234d0092,1326732,2024-07-30,2,21,2004-02-12,Полиамория и сологамия: какими бывают современные отношения,https://cuprum.media/lifestyle/romantic-relationship?from=avaapp,10308.0,2021-04-07,longread,Секс,Жизнь
30837,9609015,615c6f0cf2896444877b8b64,1430340,2024-07-30,1,44,1981-01-29,Как бороться с пигментацией кожи,https://cuprum.media/lifestyle/skin-pigmentation?from=avaapp,4227.0,2021-10-05,longread,Кожа,Жизнь
30838,9609066,654612ff65e72be96c09461b,5716,2024-07-30,1,46,1978-12-11,Как избавиться от второго подбородка,https://cuprum.media/science-answers/ubrat-vtoroy-podborodok?from=avaapp,741.0,2023-11-06,longread,"Красота,Образ жизни",Ответили по науке


In [ ]:
# оставляем только тех пользователей, у которых есть не слишком много и не слишком мало взаимодействий
def filter_for_iteration_range(df_raw, min=4, max=50):
    interaction_counts = df_raw.groupby('ehr_id')['article_id'].count()
    users_in_range = set(interaction_counts[(interaction_counts >= min) & (interaction_counts <= max)].index)
    df_filtered = data[data['ehr_id'].isin(users_in_range)]
    return df_filtered

# негерируем негативные сэмлы для работы модели
def generate_negative_samples(df, all_articles, n_negatives=3):
    negatives = []

    for user_id, group in df.groupby('ehr_id'):
        clicked = set(group['article_id'])
        not_clicked = list(all_articles - clicked)
        sampled = np.random.choice(not_clicked, size=n_negatives, replace=False)

        for art in sampled:
            negatives.append({
                "ehr_id": user_id,
                "article_id": art,
                "label": 0
            })

    return pd.DataFrame(negatives)

In [18]:
data = filter_for_iteration_range(data)
data

,id,article_id,ehr_id,created_at,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
44,3753815,60258f50e3c78375686c0c51,1132756,2023-07-01,1,48,1976-12-22,"​Как понять, что у вас паразиты",https://cuprum.media/science-answers/find-parasites?from=avaapp,5779.0,2021-02-11,longread,0,Ответили по науке
51,3754559,62e24b5be4ff39162208fd33,1169819,2023-07-01,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью
52,3754575,62e24b5be4ff39162208fd33,1169819,2023-07-01,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью
53,3754576,62e24610a9c9279df00b8673,1169819,2023-07-01,1,31,1993-07-08,"Что почитать: «Мальчик, который не переставал расти» Эдвина Кёрка",https://cuprum.media/razbor/the-boy-who-never-stopped-growing?from=avaapp,347.0,2022-07-28,longread,Книги,Разбор
54,3754581,62e0f1c5df18e4454c047292,1169819,2023-07-01,1,31,1993-07-08,«В суете нашей жизни некогда подумать о собственном здоровье»: как предотвратить инсульт,https://cuprum.media/interview/stroke-prevention?from=avaapp,892.0,2022-07-27,interview,Диагностика,Интервью
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30815,9603468,622b44d60e7b1e54fe5e5342,272451,2024-07-30,1,52,1973-01-09,Cкрининг на рак груди,https://cuprum.media/spravochnik/screening-breast-cancer-sp?from=avaapp,1072.0,2022-03-11,shortread,Скрининг,Справочник
30816,9603700,6218dd9140e8460b6d1e326b,57714,2024-07-30,1,58,1966-06-25,"Если хорошенько потрястись, это поможет снять стресс?",https://cuprum.media/proverka-sluha/shaking-off?from=avaapp,6086.0,2022-02-25,longread,Ментальное,Проверка слуха
30817,9603737,617bfecbd4f2793d703e2db2,1145846,2024-07-30,1,36,1989-05-11,"Одноразовые электронные сигареты опаснее, чем обычные вейпы?",https://cuprum.media/razbor/vape-nation?from=avaapp,11716.0,2021-10-29,longread,"Горло,Гаджеты",Разбор
30818,9603851,63e10773dbd23b1bff0b7002,263328,2024-07-30,1,31,1993-09-13,Плесень на орехах и кукурузе вызывает рак печени?,https://cuprum.media/science-answers/nuts-and-cancer?from=avaapp,1455.0,2023-02-06,longread,"Еда,Онкология,Патогены",Ответили по науке


In [20]:
## Разбиваем данные на тестовый и тренировочный сеты по времени, 80/20 процентов с разбивкой по времени
data = data.dropna(subset=['gender', 'rubric_title', 'tags', 'formats'])
data['label'] = 1
data['created_at'] = pd.to_datetime(data['created_at'])
data = data.sort_values("created_at")

cutoff_date = data['created_at'].quantile(0.8)
train_df = data[data['created_at'] <= cutoff_date]
test_df = data[data['created_at'] > cutoff_date]

data

,id,article_id,ehr_id,created_at,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title,label
44,3753815,60258f50e3c78375686c0c51,1132756,2023-07-01,1,48,1976-12-22,"​Как понять, что у вас паразиты",https://cuprum.media/science-answers/find-parasites?from=avaapp,5779.0,2021-02-11,longread,0,Ответили по науке,1
51,3754559,62e24b5be4ff39162208fd33,1169819,2023-07-01,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью,1
52,3754575,62e24b5be4ff39162208fd33,1169819,2023-07-01,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью,1
53,3754576,62e24610a9c9279df00b8673,1169819,2023-07-01,1,31,1993-07-08,"Что почитать: «Мальчик, который не переставал расти» Эдвина Кёрка",https://cuprum.media/razbor/the-boy-who-never-stopped-growing?from=avaapp,347.0,2022-07-28,longread,Книги,Разбор,1
54,3754581,62e0f1c5df18e4454c047292,1169819,2023-07-01,1,31,1993-07-08,«В суете нашей жизни некогда подумать о собственном здоровье»: как предотвратить инсульт,https://cuprum.media/interview/stroke-prevention?from=avaapp,892.0,2022-07-27,interview,Диагностика,Интервью,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30818,9603851,63e10773dbd23b1bff0b7002,263328,2024-07-30,1,31,1993-09-13,Плесень на орехах и кукурузе вызывает рак печени?,https://cuprum.media/science-answers/nuts-and-cancer?from=avaapp,1455.0,2023-02-06,longread,"Еда,Онкология,Патогены",Ответили по науке,1
30814,9602975,62012efa3b61f04c0a2f2725,1049448,2024-07-30,1,33,1991-11-30,Может ли у грибов развиваться устойчивость к лекарствам,https://cuprum.media/proverka-sluha/antifungal-resistance?from=avaapp,2686.0,2022-02-07,longread,Наука,Проверка слуха,1
30815,9603468,622b44d60e7b1e54fe5e5342,272451,2024-07-30,1,52,1973-01-09,Cкрининг на рак груди,https://cuprum.media/spravochnik/screening-breast-cancer-sp?from=avaapp,1072.0,2022-03-11,shortread,Скрининг,Справочник,1
30816,9603700,6218dd9140e8460b6d1e326b,57714,2024-07-30,1,58,1966-06-25,"Если хорошенько потрястись, это поможет снять стресс?",https://cuprum.media/proverka-sluha/shaking-off?from=avaapp,6086.0,2022-02-25,longread,Ментальное,Проверка слуха,1


In [6]:
all_articles = set(data['article_id'].unique())

train_neg = generate_negative_samples(train_df, all_articles, n_negatives=20)
train_pos = train_df.copy()
train_pos['label'] = 1

# Нужно добавить article-фичи к негативным:
article_features = data.drop_duplicates('article_id')[
    ['article_id', 'rubric_title', 'tags', 'formats', 'views']
]

train_neg = train_neg.merge(article_features, on='article_id', how='left')

# И добавить user-фичи (gender, age):
user_features = data.drop_duplicates('ehr_id')[['ehr_id', 'gender', 'age']]
train_neg = train_neg.merge(user_features, on='ehr_id', how='left')

# Объединяем:
train_df_full = pd.concat([train_pos, train_neg], ignore_index=True)


In [7]:
features = ['gender', 'age', 'rubric_title', 'tags', 'formats', 'views']
cat_features = ['gender', 'rubric_title', 'tags', 'formats']

train_df_full = train_df_full.sort_values("ehr_id")
group_sizes = train_df_full.groupby('ehr_id').size().values

train_pool = Pool(
    data=train_df_full[features],
    label=train_df_full['label'],
    group_id=train_df_full['ehr_id'],
    cat_features=cat_features
)

model = CatBoostRanker(iterations=300, learning_rate=0.1, depth=6, verbose=50)
model.fit(train_pool)


Groupwise loss function. OneHotMaxSize set to 10
0:	total: 125ms	remaining: 37.2s
50:	total: 1.99s	remaining: 9.72s
100:	total: 3.8s	remaining: 7.48s
150:	total: 5.64s	remaining: 5.57s
200:	total: 7.49s	remaining: 3.69s
250:	total: 9.32s	remaining: 1.82s
299:	total: 11.2s	remaining: 0us


In [8]:
# 1. Повторно получаем список всех статей
all_articles = set(data['article_id'].unique())

# 2. Генерируем негативные примеры
test_neg = generate_negative_samples(test_df, all_articles, n_negatives=20)

# 3. Метка для позитивных примеров
test_pos = test_df.copy()
test_pos['label'] = 1

# 4. Добавляем признаки к негативам
test_neg = test_neg.merge(article_features, on='article_id', how='left')
test_neg = test_neg.merge(user_features, on='ehr_id', how='left')

# 5. Объединяем позитивные и негативные примеры
test_df_full = pd.concat([test_pos, test_neg], ignore_index=True)

# 6. Создаём test_pool
test_df_full = test_df_full.sort_values("ehr_id")

test_pool = Pool(
    data=test_df_full[features],
    label=test_df_full['label'],
    group_id=test_df_full['ehr_id'],
    cat_features=cat_features
)


In [9]:
test_df_full['prediction'] = model.predict(test_pool)


In [10]:
def hit_rate_at_k(df, k=5):
    hits = 0
    total_users = 0

    for user, group in df.groupby('ehr_id'):
        top_k = group.sort_values('prediction', ascending=False).head(k)
        if (top_k['label'] == 1).any():
            hits += 1
        total_users += 1

    return hits / total_users


In [11]:
hr5 = hit_rate_at_k(test_df_full, k=5)
print(f"HitRate@5: {hr5:.4f}")

HitRate@5: 0.5234


In [12]:
import numpy as np

def mrr_at_k(df, k=5):
    mrr_total = 0.0
    user_count = 0

    for _, group in df.groupby('ehr_id'):
        group_sorted = group.sort_values('prediction', ascending=False).head(k)
        relevance = group_sorted['label'].values

        # Find the rank of the first relevant item
        for rank, rel in enumerate(relevance, start=1):
            if rel == 1:
                mrr_total += 1.0 / rank
                break
        user_count += 1

    return mrr_total / user_count if user_count > 0 else 0.0


def ndcg_at_k(df, k=5):
    ndcg_total = 0.0
    user_count = 0

    for _, group in df.groupby('ehr_id'):
        group_sorted = group.sort_values('prediction', ascending=False).head(k)
        relevance = group_sorted['label'].values

        dcg = 0.0
        for i, rel in enumerate(relevance):
            dcg += (2**rel - 1) / np.log2(i + 2)

        ideal_relevance = sorted(relevance, reverse=True)
        idcg = 0.0
        for i, rel in enumerate(ideal_relevance):
            idcg += (2**rel - 1) / np.log2(i + 2)

        if idcg > 0:
            ndcg_total += dcg / idcg
        else:
            ndcg_total += 0.0

        user_count += 1

    return ndcg_total / user_count if user_count > 0 else 0.0


In [15]:
mrr5 = mrr_at_k(test_df_full, k=5)
ndcg5 = ndcg_at_k(test_df_full, k=5)

print(f"MRR@5: {mrr5:.4f}")
print(f"NDCG@5: {ndcg5:.4f}")


MRR@5: 0.3001
NDCG@5: 0.3541


In [14]:
from catboost import CatBoostRanker, Pool

features = ['gender', 'age', 'rubric_title', 'tags', 'formats', 'views']
cat_features = ['gender', 'rubric_title', 'tags', 'formats']

#data = data.dropna(subset=['gender', 'rubric_title', 'tags', 'formats'])

# Удаляем строки с NaN в категориальных признаках
train_df = train_df.dropna(subset=cat_features)

# Приводим категориальные признаки к строке
for col in cat_features:
    train_df[col] = train_df[col].astype(str)

# Сортировка по group_id (обязательно для CatBoostRanker)
train_df = train_df.sort_values('ehr_id')


train_pool = Pool(data=train_df[features],
                  label=train_df['label'],
                  group_id=train_df['ehr_id'],
                  cat_features=cat_features)

model = CatBoostRanker(iterations=300, learning_rate=0.1, depth=6, verbose=50)
model.fit(train_pool)


CatBoostError: /src/catboost/catboost/libs/metrics/metric.cpp:6526: All train targets are equal

In [16]:
train_df

,id,article_id,ehr_id,created_at,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title,label
18497,6801107,624da036b341c600db685964,3670,2024-01-25,1,54,1970-09-07,Почему ребенок всегда просит макароны с сыром,https://cuprum.media/columns/pasta-and-kids?from=avaapp,3358.0,2022-04-06,interview,Питание,Блог врачей,1
18498,6801131,624dc403d6fbe97b384ccc22,3670,2024-01-25,1,54,1970-09-07,Есть ли толк от очистителей воздуха,https://cuprum.media/proverka-sluha/air-purifiers-work-or-not?from=avaapp,1797.0,2022-04-06,longread,"Экология,Вредное",Проверка слуха,1
11258,5439270,60f57272c17ff86423582032,3670,2023-10-25,1,54,1970-09-07,"Правда, что картошка — самый бесполезный овощ?",https://cuprum.media/proverka-sluha/potato-are-not-villain?from=avaapp,6682.0,2021-07-19,longread,Еда,Проверка слуха,1
11259,5439299,60f6e195a9e57f28ce4194b3,3670,2023-10-25,1,54,1970-09-07,Как поговорить с бабушкой о здоровье,https://cuprum.media/razbor/stubborn-grandmother?from=avaapp,2796.0,2021-07-20,longread,"Взрослеем как взрослые,Старший возраст,Спецпроект",Разбор,1
18470,6794430,61039e8a65fa090ddd554472,3670,2024-01-25,1,54,1970-09-07,B12 и фолиевая кислота,https://cuprum.media/spravochnik/b12-i-folievaya-kislota?from=avaapp,2269.0,2021-07-31,shortread,Обследования,Справочник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23829,7927581,625991995319175a274b95f3,1396493,2024-04-05,2,24,2000-11-04,Как еда влияет на настроение,https://cuprum.media/razbor/good-mood-food?from=avaapp,3792.0,2022-04-15,longread,Питание,Разбор,1
23839,7929756,625834f2879bd209ac6e1123,1396493,2024-04-05,2,24,2000-11-04,"Что происходит с моими руками, пока я залипаю в телефон",https://cuprum.media/ipohondrik/hand-phone?from=avaapp,8140.0,2022-04-14,longread,"Суставы,Руки",Ипохондрик,1
23830,7927589,6258368f56efd0745c2cd364,1396493,2024-04-05,2,24,2000-11-04,"Кто-то взорвал бочку с азотной кислотой и есть риск, что облако путешествует. Это правда опасно?",https://cuprum.media/proverka-sluha/nitric-acid-cloud?from=avaapp,2009.0,2022-04-14,longread,"Вредное,Яд ,Экология",Проверка слуха,1
24015,7974349,5f918991b2b1e5081913df51,1396802,2024-04-08,1,44,1981-06-14,Ночная потливость Ричарда из «Кремниевой долины»,https://cuprum.media/razbor/night-sweats?from=avaapp,1017.0,2020-10-22,longread,"Сон,Ментальное",Разбор,1


In [ ]:
train_articles = set(train_df['article_id'])
test_articles = set(test_df['article_id'])

only_in_test = test_articles - train_articles
print(f"Новых статей в тесте: {len(only_in_test)}")

Новых статей в тесте: 316


In [ ]:
article_feats_ids = set(article_features['article_id'])
missing = only_in_test - article_feats_ids
print(f"Статей без фичей: {len(missing)}")  # желательно 0


Статей без фичей: 0
